In [1]:
import numpy as np
import pandas as pd
import skyfield
from skyfield.api import load
from skyfield.positionlib import ICRF, Barycentric
import astropy
from astropy.units import deg, au, km, meter, day, minute, second
from astropy.coordinates import SkyCoord, GCRS, BarycentricMeanEcliptic, HeliocentricMeanEcliptic
import matplotlib.pyplot as plt

# MSE imports
import astro_utils
from astro_utils import jd_to_mjd, radec2dir, radec_diff, qv2radec, qvrel2radec

In [2]:
light_speed = astropy.constants.c.to(au / minute)
light_speed

<Quantity 0.12023933 AU / min>

### Observation of Earth and Mars according to JPL

In [84]:
def add_mjd_col(df):
    """Add a new column to a DataFrame with the MJD"""
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5).astype(np.int32)
    # Order columns
    columns = df.columns.tolist()
    n_col = len(columns)
    columns_new = columns[n_col-1:n_col] + columns[1:n_col-1]
    df = df[columns_new]
    return df

In [85]:
# Load the JPL position of Earth as CSV
df_earth = pd.read_csv('../data/jpl/testing/vectors-earth.txt', index_col=False)
# Add column for mjd
df_earth = add_mjd_col(df_earth)

# Display the dataframe
df_earth

,mjd,CalendarDate,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,55197,A.D. 2010-Jan-01 00:00:00.0000,-0.179765,0.970347,-0.000017,-0.017202,-0.003148,8.961125e-07,0.005700,0.986858,0.000038
1,55198,A.D. 2010-Jan-02 00:00:00.0000,-0.196939,0.967049,-0.000017,-0.017145,-0.003447,9.036109e-07,0.005700,0.986899,0.000044
2,55199,A.D. 2010-Jan-03 00:00:00.0000,-0.214053,0.963453,-0.000016,-0.017083,-0.003745,8.653246e-07,0.005700,0.986945,0.000049
3,55200,A.D. 2010-Jan-04 00:00:00.0000,-0.231103,0.959559,-0.000015,-0.017017,-0.004042,7.855759e-07,0.005700,0.986997,0.000054
4,55201,A.D. 2010-Jan-05 00:00:00.0000,-0.248085,0.955369,-0.000014,-0.016945,-0.004339,6.725245e-07,0.005701,0.987054,0.000059
...,...,...,...,...,...,...,...,...,...,...,...
3648,58845,A.D. 2019-Dec-28 00:00:00.0000,-0.100787,0.986067,-0.000022,-0.017416,-0.001767,9.809367e-07,0.005725,0.991204,0.000013
3649,58846,A.D. 2019-Dec-29 00:00:00.0000,-0.118187,0.984147,-0.000021,-0.017382,-0.002073,9.327289e-07,0.005725,0.991218,0.000015
3650,58847,A.D. 2019-Dec-30 00:00:00.0000,-0.135550,0.981922,-0.000020,-0.017343,-0.002377,8.590659e-07,0.005725,0.991234,0.000017
3651,58848,A.D. 2019-Dec-31 00:00:00.0000,-0.152871,0.979393,-0.000019,-0.017298,-0.002681,7.650397e-07,0.005725,0.991251,0.000019


In [86]:
# Load the JPL position of Mars as CSV
df_mars = pd.read_csv('../data/jpl/testing/vectors-mars.txt', index_col=False)
# Add column for mjd
df_mars = add_mjd_col(df_mars)

# Display the dataframe
df_mars

,mjd,CalendarDate,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,55197,A.D. 2010-Jan-01 00:00:00.0000,-0.733418,1.457212,0.048394,-0.011980,-0.005093,0.000188,0.009426,1.632088,0.000842
1,55198,A.D. 2010-Jan-02 00:00:00.0000,-0.745374,1.452070,0.048580,-0.011930,-0.005192,0.000184,0.009431,1.632927,0.000834
2,55199,A.D. 2010-Jan-03 00:00:00.0000,-0.757278,1.446828,0.048763,-0.011879,-0.005291,0.000181,0.009436,1.633756,0.000826
3,55200,A.D. 2010-Jan-04 00:00:00.0000,-0.769131,1.441488,0.048942,-0.011827,-0.005390,0.000178,0.009441,1.634578,0.000817
4,55201,A.D. 2010-Jan-05 00:00:00.0000,-0.780931,1.436049,0.049118,-0.011774,-0.005488,0.000174,0.009445,1.635391,0.000809
...,...,...,...,...,...,...,...,...,...,...,...
3648,58845,A.D. 2019-Dec-28 00:00:00.0000,-1.356374,-0.836106,0.015533,0.007920,-0.010678,-0.000418,0.009203,1.593445,-0.001143
3649,58846,A.D. 2019-Dec-29 00:00:00.0000,-1.348404,-0.846754,0.015114,0.008019,-0.010616,-0.000419,0.009196,1.592299,-0.001149
3650,58847,A.D. 2019-Dec-30 00:00:00.0000,-1.340336,-0.857339,0.014695,0.008117,-0.010553,-0.000420,0.009190,1.591146,-0.001155
3651,58848,A.D. 2019-Dec-31 00:00:00.0000,-1.332170,-0.867860,0.014274,0.008215,-0.010490,-0.000421,0.009183,1.589988,-0.001161


In [5]:
# Load the JPL RA and DEC for Mars as fixed width file
df_obs = pd.read_fwf('../data/jpl/testing/observe-mars-earth-geocenter.txt')

# Display the dataframe
df_obs

,Date,Time,JulianDate,RA,DEC,RA_apparent,DEC_apparent delta,delta_dot,light_time
0,2010-Jan-01,00:00,2455197.5,142.327061,18.799029,142.475968,18.752303941 0.73883173669463,-8.970408,6.144676
1,2010-Jan-02,00:00,2455198.5,142.179197,18.888379,142.328371,18.841710905 0.73372184383589,-8.724067,6.102178
2,2010-Jan-03,00:00,2455199.5,142.017468,18.981461,142.166897,18.934867206 0.72875633894059,-8.470333,6.060882
3,2010-Jan-04,00:00,2455200.5,141.841890,19.078173,141.991560,19.031670832 0.72393956594361,-8.208951,6.020822
4,2010-Jan-05,00:00,2455201.5,141.652504,19.178403,141.802402,19.132004243 0.71927600922442,-7.939695,5.982036
...,...,...,...,...,...,...,...,...,...
3648,2019-Dec-28,00:00,2458845.5,233.190100,-18.727810,233.467326,-18.791714023 2.21286843425630,-12.191696,18.403866
3649,2019-Dec-29,00:00,2458846.5,233.876730,-18.895847,234.154585,-18.958738129 2.20580870839279,-12.255895,18.345152
3650,2019-Dec-30,00:00,2458847.5,234.565048,-19.061459,234.843518,-19.123323188 2.19871232427735,-12.318649,18.286133
3651,2019-Dec-31,00:00,2458848.5,235.255049,-19.224614,235.534120,-19.285436340 2.19158011449378,-12.379967,18.226816


In [ ]:
# Extract position and velocity of earth from df_earth
q_earth_jpl = np.array([df_earth.X.values, df_earth.Y.values, df_earth.Z.values]) * au
v_earth_jpl = np.array([df_earth.VX.values, df_earth.VY.values, df_earth.VZ.values]) * au / day

# Extract position of mars from df_mars
q_mars_jpl = np.array([df_mars.X.values, df_mars.Y.values, df_mars.Z.values]) * au
v_mars_jpl = np.array([df_mars.VX.values, df_mars.VY.values, df_mars.VZ.values]) * au / day

# Extract obstime_jd, ra, and dec from DataFrame
obstime_jd = df_obs.JulianDate.values
ra_jpl = df_obs.RA.values
dec_jpl = df_obs.DEC.values

# Vector of observation times in MJD format
obstime_mjd = jd_to_mjd(obstime_jd)

### Position & Observation of Earth and Mars according to Skyfield

In [ ]:
# Manually load planetary positions using de435
planets_sf = load('../data/jpl/ephemeris/de435.bsp')
earth_sf = planets_sf['earth']
mars_sf = planets_sf['mars barycenter']

# load timescale
ts = load.timescale()

# Generate vector of observation times in Skyfield format
obstime_sf = ts.tt_jd(obstime_jd)

In [ ]:
# Observe mars from earth with Skyfield
obs_sf = earth_sf.at(obstime_sf).observe(mars_sf)

# Build Skyfield angle arrays (RA, DEC) and distance array (delta)
ra_sf_aa, dec_sf_aa, delta_sf_da = obs_sf.radec()

# Extract degrees and AU to get plain arrays
ra_sf = ra_sf_aa._degrees
dec_sf = dec_sf_aa._degrees
delta_sf = delta_sf_da.au

In [ ]:
# Load planetary positions and velocities by querying the Skyfield JPL ephemeris interface
# Create them as arrays with bundled astropy units of au and km / second

# Earth
q_earth_sf = earth_sf.at(obstime_sf).ecliptic_position().au * au
v_earth_sf = earth_sf.at(obstime_sf).ecliptic_velocity().km_per_s * km / second

# Mars
q_mars_sf = mars_sf.at(obstime_sf).ecliptic_position().au * au
v_mars_sf = mars_sf.at(obstime_sf).ecliptic_velocity().km_per_s * km / second

In [ ]:
# Demonstrate that q_earth_sf is the same as q_earth_jpl
q_earth_eps = np.mean(np.linalg.norm(q_earth_sf - q_earth_jpl, axis=0))
v_earth_eps = np.mean(np.linalg.norm(v_earth_sf - v_earth_jpl, axis=0))
q_mars_eps = np.mean(np.linalg.norm(q_mars_sf - q_mars_jpl, axis=0))
v_mars_eps = np.mean(np.linalg.norm(v_mars_sf - v_mars_jpl, axis=0))

# Report
print('Difference between Skyfield (JPL ephem) and Horizons download:')
print(f'q_earth : {q_earth_eps:5.3e} au')
print(f'v_earth : {v_earth_eps:5.3e} au / day')
print(f'q_mars  : {q_mars_eps:5.3e} au')
print(f'v_mars  : {v_mars_eps:5.3e} au / day')

In [ ]:
# Alias q_earth, v_earth, q_mars, v_mars to the Skyfield interpolation for legibility
q_earth = q_earth_sf
v_earth = v_earth_sf
q_mars = q_mars_sf
v_mars = v_mars_sf

### Compare Skyfield vs JPL

In [ ]:
# Compute difference in angles
diff_sf = radec_diff('JPL', 'Skyfield', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_sf, dec2=dec_sf, 
                     obstime_mjd=obstime_mjd, verbose=False)
diff_mean = np.mean(diff_sf)
diff_median = np.median(diff_sf)
diff_max = np.max(diff_sf)

# Report results
print(f'Mean Angle Difference: JPL vs. Skyfield')
print(f'Mean  : {diff_mean:5.3f} seconds')
print(f'Median: {diff_median:5.3f} seconds')
print(f'Max   : {diff_max:5.3f} seconds')

### Observation of Mars Using MSE Function qv2radec and Skyfield Positions

In [ ]:
# Calculate light time from mars to earth
dist_e2m = np.linalg.norm(q_mars - q_earth, axis=0)* au
light_time = dist_e2m / light_speed

# Adjustment of mars position for light time
dq_mars_lt = v_mars.to(au / minute) * light_time

In [ ]:
# Use simple function that accepts only the position and velocity of the observed body
ra_mse1, dec_mse1, r_mse1 = qv2radec(q=q_mars, v=v_mars, mjd=obstime_mjd, frame=BarycentricMeanEcliptic)

In [ ]:
# Compute difference in angles
diff_mse1 = radec_diff('JPL', 'MSE', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_mse1, dec2=dec_mse1, 
                     obstime_mjd=obstime_mjd, verbose=False)
diff_mean = np.mean(diff_mse1)
diff_median = np.median(diff_mse1)
diff_max = np.max(diff_mse1)

# Report results
print(f'Mean Angle Difference: JPL vs. MSE qv2radec')
print(f'Mean  : {diff_mean:6.3f} seconds')
print(f'Median: {diff_median:6.3f} seconds')
print(f'Max   : {diff_max:6.3f} seconds')

In [ ]:
# Day with biggest error
idx = np.argmax(diff_mse1)
print(f'Date with largest error:')
print(f'index    = {idx}')
print(f'MJD      = {obstime_mjd[idx]}')
print(f'Date     = {astro_utils.mjd_to_date(obstime_mjd[idx])}')
print()
print('q_earth   = ', q_earth[:, idx])
print('q_mars    = ', q_mars[:, idx])
print()
print(f'ra_mse1  = {ra_mse1[idx]:10.6f}')
print(f'ra_jpl   = {ra_jpl[idx]:10.6f}')
print(f'ra diff  = {ra_mse1[idx]-ra_jpl[idx]:10.6f}')
print()
print(f'dec_mse1 = {dec_mse1[idx]:10.6f}')
print(f'dec_jpl  = {dec_jpl[idx]:10.6f}')
print(f'dec diff = {dec_mse1[idx]-dec_jpl[idx]:10.6f}')
print()
print(f'AngleDiff= {diff_mse1[idx]:7.3f}    seconds')

In [ ]:
_ = plt.plot(diff_mse1)
plt.show()

### Observation of Mars Using MSE Function qvrel2radec and Skyfield Positions

In [ ]:
# Use advanced function that  accepts position and velocity of body and earth
ra_mse2, dec_mse2, r_mse2 = qvrel2radec(q_body=q_mars, v_body=v_mars, 
                                        q_earth=q_earth, v_earth=v_earth,
                                        mjd=obstime_mjd, frame=BarycentricMeanEcliptic)

In [ ]:
# Compute difference in angles
diff_mse2 = radec_diff('JPL', 'MSE', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_mse2, dec2=dec_mse2, 
                     obstime_mjd=obstime_mjd, verbose=False)
diff_mean = np.mean(diff_mse2)
diff_median = np.median(diff_mse2)
diff_max = np.max(diff_mse2)

# Report results
print(f'Mean Angle Difference: JPL vs. MSE qvrel2radec')
print(f'Mean  : {diff_mean:6.3f} seconds')
print(f'Median: {diff_median:6.3f} seconds')
print(f'Max   : {diff_max:6.3f} seconds')

### Vectors of First 16 Asteroids from JPL

In [87]:
def add_mjd_astnum_col(df, asteroid_num: int):
    """Add a new columns to a DataFrame with the Asteroid number and MJD"""
    # Add column for asteroid_num
    df['asteroid_num'] = asteroid_num
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5).astype(np.int32)
    # Order columns
    columns = df.columns.tolist()
    n_col = len(columns)
    columns_new = columns[n_col-2:n_col] + columns[2:n_col-2]
    df = df[columns_new]
    return df

In [88]:
# List of dataframes; one per asteroid
df_ast_list = []

# Load the JPL position of asteroids one at a time
for j in range(1, 17):
    # print(j)
    df_ast_j = pd.read_csv(f'../data/jpl/testing/vectors-asteroid-{j:03d}.txt', index_col=False)
    # Add columns for the asteroid_num and mjd
    df_ast_j = add_mjd_astnum_col(df_ast_j, j)
    # Add this to list of frames
    df_ast_list.append(df_ast_j)

# Concatenate dataframes
df_ast = pd.concat(df_ast_list)

In [89]:
 df_ast

,asteroid_num,mjd,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,1,55197,-1.660333,-2.123236,0.238962,0.007615,-0.007150,-0.001627,0.015628,2.705909,0.000794
1,1,55198,-1.652706,-2.130370,0.237334,0.007640,-0.007118,-0.001630,0.015633,2.706703,0.000794
2,1,55199,-1.645053,-2.137472,0.235702,0.007665,-0.007086,-0.001634,0.015637,2.707497,0.000795
3,1,55200,-1.637376,-2.144542,0.234066,0.007689,-0.007054,-0.001637,0.015642,2.708293,0.000796
4,1,55201,-1.629675,-2.151580,0.232427,0.007713,-0.007022,-0.001641,0.015646,2.709088,0.000796
...,...,...,...,...,...,...,...,...,...,...,...
3648,16,58845,2.517677,-0.513079,-0.043698,0.001590,0.011210,-0.000568,0.014842,2.569797,-0.000671
3649,16,58846,2.519245,-0.501865,-0.044266,0.001546,0.011219,-0.000568,0.014838,2.569129,-0.000665
3650,16,58847,2.520770,-0.490641,-0.044834,0.001503,0.011228,-0.000567,0.014834,2.568466,-0.000660
3651,16,58848,2.522251,-0.479409,-0.045400,0.001459,0.011236,-0.000566,0.014830,2.567809,-0.000655


### Observations of First 16 Asteroids from JPL

In [99]:
# List of dataframes; one per asteroid
df_obs_list = []

j=1
df_obs_j = pd.read_fwf(f'../data/jpl/testing/observer-asteroid-{j:03d}-earth-geocenter.txt', index_col=False)
# Add columns for the asteroid_num and mjd
df_obs_j = add_mjd_astnum_col(df_obs_j, j)
# Add this to list of frames
df_obs_list.append(df_ast_j)


In [98]:
df_obs_j

,asteroid_num,mjd,JulianDate,RA,DEC,RA_apparent,DEC_apparent delta,delta_dot,light_time
0,1,55197,2455197.5,243.215442,-17.105913,243.358581,-17.131843567 3.43787712513684,-12.468091,28.591952
1,1,55198,2455198.5,243.625145,-17.196033,243.768548,-17.221644876 3.43061845196470,-12.668422,28.531584
2,1,55199,2455199.5,244.034084,-17.284935,244.177730,-17.310227391 3.42324422398770,-12.868277,28.470254
3,1,55200,2455200.5,244.442231,-17.372621,244.586099,-17.397588717 3.41575469301738,-13.067723,28.407966
4,1,55201,2455201.5,244.849560,-17.459094,244.993632,-17.483728144 3.40815009448535,-13.266749,28.344720
...,...,...,...,...,...,...,...,...,...
3648,1,58845,2458845.5,287.919841,-26.370367,288.216847,-26.336700061 3.87255763303848,5.256575,32.207080
3649,1,58846,2458846.5,288.363453,-26.341715,288.660325,-26.307263794 3.87551888026995,4.997917,32.231708
3650,1,58847,2458847.5,288.807029,-26.311943,289.103749,-26.276711753 3.87833066727208,4.739027,32.255093
3651,1,58848,2458848.5,289.250537,-26.281056,289.547090,-26.245048584 3.88099289014077,4.480007,32.277234


In [ ]:
read_fwf